In [115]:
import numpy as np
import pandas as pd

def sigmoid(a):
    return (1/(1+(np.exp(-a))))

def sigmoidDerivative(a):
    return np.multiply(sigmoid(a), (1-sigmoid(a)))   

class NeuralNetwork:
    def __init__(self, inputNos, outputNos, totalLayers, neuronNos):
        self.inputNos = inputNos
        self.outputNos = outputNos
        self.totalLayers = totalLayers
        self.hiddenLayers = totalLayers-2
        self.neuronNos = neuronNos

        layers = [(inputNos)]
        for i in range(0, totalLayers-2):
            layers.append(neuronNos)
        layers.append(outputNos)

        self.bias = [np.random.rand(x)/100 for x in layers[1:]]
        self.weights = [np.random.rand(y,x)/100 for (x,y) in zip(layers[:-1], layers[1:])]
        self.outputs = [np.zeros(x) for x in layers]
        self.activationDerivative = [np.zeros(x) for x in layers]

    def feedforward(self, inputVector):
        self.outputs[0] = inputVector

        for i in range(1, self.totalLayers):
            temp = np.dot(self.weights[i-1], self.outputs[i-1]) + self.bias[i-1]
            self.activationDerivative[i] = sigmoidDerivative(temp)
            self.outputs[i] = (sigmoid(temp))

    def backprop(self, inputVector, outputVectors, learningRate):
        del_weights = [np.zeros(w.shape) for w in self.weights]
        del_bias = [np.zeros(b.shape) for b in self.bias]

        delta = np.multiply((outputVectors-self.outputs[-1]), self.activationDerivative[-1])
        del_bias[-1] = delta
        del_weights[-1] = np.dot(np.matrix(delta).transpose(), np.matrix(self.outputs[-2]))
        
        for l in range(2, self.totalLayers):
                delta = np.multiply((np.dot(self.weights[-l+1].transpose(), delta)), self.activationDerivative[-l])
                del_bias[-l] = delta
                del_weights[-l] = np.dot(np.matrix(delta).transpose(), np.matrix(self.outputs[-l-1]))

        for i in range(0, len(self.weights)):
            self.weights[i] += learningRate*del_weights[i]
        for i in range(0, len(self.bias)):
            self.bias[i] += learningRate*del_bias[i]

    def iteration(self, inputVectors, outputVectors, learningRate, nos):
        for i in range(0, nos):
            for j in range(0, len(inputVectors)):
                self.feedforward(inputVectors[j])
                self.backprop(inputVectors[j], outputVectors[j], learningRate)

In [116]:

data = pd.read_csv(r'/home/arnavgupta829/Desktop/ELL409/Assignment1/mnist_train_m.csv')
raw = []
for i in data.columns:
    raw.append(data[i])
raw_np = np.asarray(raw)
raw_np = raw_np.transpose()
# np.random.shuffle(raw_np)
inputVectors = raw_np[0:100, 1:785]
inputVectors = (inputVectors-np.mean(inputVectors))/256
outputValues = []
outputCV = []
input_cv = raw_np[4000:4100, 1:785]
input_cv = (input_cv - np.mean(input_cv))/256
for i in range(0, len(inputVectors)):
    outputValues.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    outputValues[i][raw_np[i][0]] = 1
for i in range(4000, 4100):
    outputCV.append([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    outputCV[i-4000][raw_np[i][0]] = 1
output_cvVectors = np.asarray(outputCV)
outputVectors = np.asarray(outputValues)

nn = NeuralNetwork(784, 10, 3, 256)
print("Iterating-------------------------------------")
nn.iteration(inputVectors, outputVectors, 1, 100)
print("Done Iterating--------------------------------")


Iterating-------------------------------------
Done Iterating--------------------------------


In [114]:
import sklearn.metrics as skm
target_names = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']
print("Tring ff on cross validation set")
test_outputs = []
for i in range(0, len(input_cv)):
    nn.feedforward(input_cv[i])
    outList = nn.outputs[-1]
    maxVal = 0
    for j in range(0, len(outList)):
        if(outList[j]>outList[maxVal]):
            maxVal = j
    test_outputs.append(maxVal)
print(skm.classification_report(test_outputs, raw_np[4000:4100, 0:1], target_names=target_names))
print(skm.f1_score(raw_np[4000:4100, 0:1], test_outputs, average="macro"))
for i in range(0, len(test_outputs)):
    print("(",test_outputs[i],", ",raw_np[i+4000][0],")")

Tring ff on cross validation set
              precision    recall  f1-score   support

     class_0       1.00      0.86      0.92         7
     class_1       1.00      0.71      0.83         7
     class_2       0.91      1.00      0.95        10
     class_3       0.92      0.92      0.92        12
     class_4       0.82      0.82      0.82        11
     class_5       1.00      0.71      0.83         7
     class_6       0.92      1.00      0.96        12
     class_7       1.00      0.83      0.91        18
     class_8       0.75      0.90      0.82        10
     class_9       0.40      0.67      0.50         6

    accuracy                           0.86       100
   macro avg       0.87      0.84      0.85       100
weighted avg       0.89      0.86      0.87       100

0.8464245754245754
( 8 ,  8 )
( 3 ,  3 )
( 7 ,  7 )
( 4 ,  9 )
( 7 ,  7 )
( 8 ,  8 )
( 8 ,  8 )
( 0 ,  0 )
( 2 ,  2 )
( 8 ,  8 )
( 2 ,  2 )
( 5 ,  5 )
( 1 ,  1 )
( 7 ,  7 )
( 6 ,  6 )
( 2 ,  2 )
( 7 ,  7 )
( 

In [73]:
import sklearn.metrics as skm
print("Trying ff after iteration---------------------")
test_data = pd.read_csv(r'/home/arnavgupta829/Desktop/ELL409/Assignment1/mnist_test.csv')
raw_test = []
for i in test_data.columns:
    raw_test.append(test_data[i])
raw_test_np = np.asarray(raw_test)
raw_test_np = raw_test_np.transpose()
raw_test_np = (raw_test_np - np.mean(raw_test_np))/256
test_outputs = []
for i in range(0, len(raw_test_np)):
    nn.feedforward(raw_test_np[i])
    outList = nn.outputs[-1]
    maxVal = 0
    for j in range(0, len(outList)):
        if(outList[j]>outList[maxVal]):
            maxVal = j
    test_outputs.append(maxVal)
print(test_outputs)

s = "\n".join([str(x) for x in test_outputs])
with open("submission.csv", "w+") as f:
    f.write(s)

Trying ff after iteration---------------------
[1, 5, 2, 5, 4, 7, 0, 4, 2, 3, 4, 6, 7, 6, 5, 7, 2, 3, 8, 6, 1, 6, 6, 8, 5, 0, 4, 0, 1, 6, 9, 3, 5, 8, 0, 8, 8, 7, 6, 5, 4, 7, 2, 4, 5, 0, 9, 3, 7, 7, 3, 8, 7, 4, 4, 7, 5, 3, 1, 2, 1, 1, 7, 2, 6, 5, 3, 0, 8, 4, 3, 9, 6, 1, 7, 9, 6, 6, 5, 0, 3, 8, 0, 1, 9, 6, 0, 3, 5, 1, 8, 5, 4, 2, 7, 4, 6, 5, 3, 9, 6, 5, 9, 6, 5, 0, 5, 8, 5, 2, 4, 0, 7, 5, 5, 0, 6, 4, 6, 5, 2, 4, 5, 2, 0, 4, 8, 8, 0, 9, 7, 5, 2, 8, 0, 2, 9, 0, 7, 7, 3, 8, 8, 0, 6, 8, 5, 2, 1, 3, 8, 4, 9, 3, 7, 9, 5, 0, 3, 6, 1, 9, 5, 1, 3, 0, 8, 0, 5, 0, 3, 6, 2, 3, 7, 1, 1, 3, 2, 7, 2, 6, 9, 7, 6, 2, 9, 6, 2, 4, 6, 9, 2, 1, 1, 0, 5, 8, 7, 1, 0, 2, 9, 4, 0, 7, 5, 3, 6, 3, 9, 3, 3, 9, 1, 3, 1, 7, 4, 3, 2, 0, 9, 3, 9, 8, 5, 6, 9, 7, 2, 0, 8, 7, 5, 8, 5, 4, 1, 1, 6, 0, 2, 9, 9, 0, 7, 2, 0, 7, 7, 0, 7, 4, 2, 7, 5, 0, 4, 5, 8, 4, 1, 8, 6, 4, 7, 0, 1, 3, 5, 1, 0, 9, 4, 1, 4, 1, 0, 6, 8, 6, 7, 2, 0, 4, 3, 0, 5, 3, 3, 6, 0, 1, 8, 8, 0, 8, 3, 2, 3, 3, 0, 5, 1, 1, 8, 7, 3, 5, 6, 4, 2, 3, 2, 3, 4, 1